# calcium silicateの物性値 
2023 HAMテスト用
参考：https://www.casiplus.de/technische-daten/

In [1]:
module calcium_silicate

include("../function/vapour.jl")

################################
### 基礎物性 ###
################################

# 空隙率
const psi = 0.893

# 材料密度
const row = 269.8 #kg/m3 

################################
# 水の密度
const roww = 1000.0 #kg/m3
#理想気体定数
const R = 8.314 # J/(mol K)
# 水のモル質量
const Mw = 0.018 # kg/mol
# 水蒸気のガス定数
#Rv = R / Mw # J/(kg K)
# 水の熱容量
const croww = 1000.0 * 4.18605e+3
# miu ⇒　rh の変換係数
function convertMiu2RH( ;temp::Float64, miu::Float64 );
    return exp( miu / Rv / temp )
end
convertMiu2RH( temp::Float64, miu::Float64 ) = convertMiu2RH( temp=temp, miu=miu )

################################
### 容量系 ###
################################

##########################################
# 比熱 specific heat capacity
const C = 1160.0 #J/kgK 

# 熱容量
get_crow( ;phi::Float64 ) = C * row + croww * phi
get_crow( cell ) = get_crow( phi = get_phi( cell ) )

##########################################
# 水分特性
# van-Genuchten用情報
include("./van_genuchten.jl")
include("./Boltzmann_transformation.jl")

BeSand_vG = van_Genuchten.vG_parameter( 
    0.007143, #α
    2.0, #n
    0.5, #m
    0.5 #l
)

# 含水率
get_phi( ;miu::Float64 ) = van_Genuchten.get_phi( vG = BeSand_vG, miu = miu, phimax = psi )
get_phi( cell ) = van_Genuchten.get_phi( vG = BeSand_vG, miu = cell.miu, phimax = psi )

# 含水率の水分化学ポテンシャル微分
get_dphi( ;miu::Float64 ) = van_Genuchten.get_dphi( vG = BeSand_vG, miu = miu, phimax = psi )
get_dphi( cell ) = get_dphi( miu = cell.miu )

# 含水率から水分化学ポテンシャルの算出
get_miu_by_phi( ;phi::Float64 ) = van_Genuchten.get_miu(vG = BeSand_vG, phi = phi, phimax = psi )
get_miu_by_phi( cell ) = get_miu_by_phi( phi = cell.phi )

################################
### 移動特性 ###
################################
# 熱伝導率
const lam = 0.767
# 湿気依存
get_lam( ;phi::Float64 ) = lam #+ 1.7227 * phi
get_lam( cell ) = get_lam( phi = get_phi( cell ) )

##########################################
# 飽和時の透水係数
const Ksat = 1.385e-5

## 透水係数（変換後）
get_dw( ;miu::Float64 ) = Ksat * van_Genuchten.get_kl( vG = BeSand_vG, miu = miu ) 
get_dw( cell ) = get_dw( ;miu = cell.miu )

###########################################
# 大気中における水蒸気の拡散係数
δ(temp, Pa) = 2.0*10-7 * temp^0.81 / Pa
const μ_value = 3.0

# 透湿率（湿気伝導率）
function get_dp( ;temp::Float64, phi::Float64 );
    return δ(293.15, 101325.0) * μ_value
end
get_dp( cell ) = get_dp( temp = cell.temp, phi = get_phi( cell ) )

end

Main.calcium_silicate

### Example

In [2]:
mutable struct test_calcium_silicate
    temp::Float64
    miu::Float64
end

In [3]:
calcium_silicate_hygro = test_calcium_silicate( 293.15, 8.314 / 0.018 * 293.15 * log( 0.7 ) )

test_calcium_silicate(293.15, -48294.76033929033)

In [4]:
calcium_silicate.get_crow( calcium_silicate_hygro )

323804.10938239406

In [5]:
calcium_silicate.get_lam( calcium_silicate_hygro ) 

0.767

In [6]:
calcium_silicate.get_phi( calcium_silicate_hygro )

0.0025886239730519393

In [8]:
calcium_silicate.get_phi( miu = -0.00000001 )

0.893

In [9]:
calcium_silicate.get_miu_by_phi( phi = calcium_silicate.get_phi( calcium_silicate_hygro ) )

-48294.76033929033

In [10]:
calcium_silicate.get_dphi( calcium_silicate_hygro )

5.3600063498106976e-8

In [20]:
calcium_silicate.get_miu_by_phi( phi = 0.1 )

-1242.3116782234763

In [21]:
calcium_silicate.get_dw( calcium_silicate_hygro )

1.316347040846987e-17

In [16]:
calcium_silicate.get_dp( calcium_silicate_hygro )

59.97935354470674